# LPAR Topology Report

 Select a **_Dataset_** to request data from  
**_(If the text field to select a Dataframe does not show up after a few seconds or you want to reset the notebook, restart the Kernel using the `>>`(Restart and run all) button above)_**

In [ ]:
import smfexplorer
import pandas as pd
import ipywidgets as widgets
import plotly as pl
import numpy as np

from smfexplorer.fields import SMF99S12 as F12
from smfexplorer.fields import SMF99S14
from smfexplorer.core.expressions import ASC
from smfexplorer.util import jupyter
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import Markdown, display, Javascript, HTML
from ipywidgets import VBox, interact, Button, Output
import plotly.express as px
from smfexplorer.error import EmptyDataSetError

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

ctx = smfexplorer.new_context()
config_widget = jupyter.ConfigWidget(ctx)
display(config_widget)

selection_widget_topo = widgets.ToggleButtons()
selection_widget_date = widgets.ToggleButtons()
selection_widget_color = widgets.ToggleButtons()
selection_widget_time = widgets.Text()


@config_widget.register_output(name="df_topo", system_topo=selection_widget_topo)
def fetch_topo_df(dsn, **kwds):
    if dsn is None:
        return
    try:
        display(Markdown(f"# Topology "))
        df_topo = ctx.samples.topology().run()
        df_topo = df_topo.rename(
            columns={
                "timestamp": "Timestamp",
                "sid": "System",
                "hd_topochg_cpu_index": "Topology Index",
                "processor": "Processor Type",
                "cp_cputype": "CPU Type",
                "vcm_lparphysprocshr": "LPAR Share",
                "speed_change": "speed",
                "honor_priority_change": "honor priority",
                "topology_change": "topology",
                "affinity_nodes_rebuild": "affinity nodes",
                "mpwq_affinity_node": "Affinity Node",
            }
        )

        unique_system = np.sort(df_topo["System"].unique())

        selection_widget_topo.options = unique_system

        if unique_system.size > 1:
            display(Markdown("# Select system for analysis"))
            display(selection_widget_topo)

        else:
            display(Markdown(f"## Using data of system {unique_system[0]}"))

        selection_widget_topo.value = unique_system[0]

        if len(unique_system) > 1:
            display(selection_widget_topo)

        return df_topo

    except EmptyDataSetError:
        display(Markdown("### Data set does not contain SMF99 Subtype 14 data"))


@fetch_topo_df.register_output(name="topo", date=selection_widget_date)
def dataframe_data(df_topo, system_topo, **kwds):
    global topo

    if df_topo is None:
        return
    topo = df_topo[df_topo["System"] == system_topo]
    topo.reset_index(inplace=True, drop=True)

    timestamps_aval = topo["Timestamp"].unique()
    rows = []
    for time_point in timestamps_aval:
        iterator = 0
        df_tmp = topo[topo["Timestamp"] == time_point]
        buckets = len(df_tmp["Topology Index"].unique())
        if df_tmp.shape[0] % buckets == 0:
            p = int(df_tmp.shape[0] / buckets)
            dpoints = df_tmp.values.reshape(p, -1, df_tmp.shape[1])
            for point in dpoints:
                if iterator < buckets:
                    rows.append(point[iterator])
                    iterator = iterator + 1
    tmp = pd.DataFrame(rows, columns=topo.columns)
    topo = tmp
    date_aval = [
        date.strftime("%m-%d-%y ") for date in topo["Timestamp"].dt.date.unique()
    ]
    selection_widget_date.options = date_aval
    if np.size(date_aval) == 1:
        display(Markdown(f"**Avaliable date:** {date_aval[0]}"))

    else:
        display(Markdown("# Select date for analysis"))
        display(selection_widget_date)

    # selection_widget_date.value = date_aval[0]

    return topo


@dataframe_data.register_output(name="topo", time=selection_widget_time)
def dataframe_time(topo, date, **kwds):
    if topo is None:
        return

    time_placeholder = pd.to_datetime(topo["Timestamp"][0]).strftime("%H:%M:%S")
    selection_widget_time.placeholder = time_placeholder
    topo["date"] = topo["Timestamp"].dt.strftime("%m-%d-%y ")
    topo["time"] = topo["Timestamp"].dt.strftime("%H:%M:%S")
    topo = topo[topo["date"] == date]
    topo.reset_index(inplace=True, drop=True)
    if (len(topo.index) != 0) and len(topo.index) % len(topo["Topology Index"].unique()) == 0:
        entry_time = topo["Timestamp"][0]
        topo_comp = topo.drop(columns=["Timestamp", "date", "time"])
        topo_comp = topo_comp.drop_duplicates()
        if len(topo_comp.index) == len(topo["Topology Index"].unique()):
            topo = topo_comp

        else:
            display(Markdown("# Select time for analysis"))
            display(selection_widget_time)
    else:
        display(Markdown("# Select time for analysis"))
        display(selection_widget_time)

    selection_widget_time.value = time_placeholder

    return topo


@dataframe_time.register_output(
    name="df_filtered", time=selection_widget_time, coloring=selection_widget_color
)
def dataframe_topo(topo, time, **kwds):
    global df_filtered

    if topo is None:
        return

    time_selection = topo[topo["time"] >= time].reset_index()

    if time_selection.empty:
        time_selection = topo[topo["time"] <= time].reset_index()["Timestamp"].iloc[-1]
    else:
        time_selection = time_selection["Timestamp"][0]
    display(
        Markdown(
            f"Selected timestamp for analysis is: {time_selection.strftime('%m-%d-%y %H:%M:%S')}"
        )
    )
    df_filtered = topo[topo["Timestamp"] == time_selection].copy(deep=True)
    colorings = ["CPU Type", "Affinity Node", "Processor Type"]
    selection_widget_color.options = colorings
    display(Markdown("## Select color diferentiation"))
    display(selection_widget_color)
    selection_widget_color.value = colorings[0]

    return df_filtered


@dataframe_topo.register_output()
def topology_viz(df_filtered, coloring, **kwds):
    if df_filtered is None:
        return
    colors = [
        "#dee2ff",
        "#edf2fb",
        "#efd3d7",
        "#F8ECEE",
        "#EAE6ED",
        "#E4F2C2",
        "#F0F7E0",
        "#FFF5CC",
        "#FFFAE6",
    ]
    changes = ["speed", "honor priority", "topology", "affinity nodes"]
    STISI = topo["cp_ci_nlinuse"].unique()[0]

    def label(row):
        set_label = (
            "<b>System:</b> "
            + row["System"]
            + "<br><b>Affinity Node:</b> "
            + str(row["Affinity Node"])
            + "<br><b>CPU:</b> "
            + row["CPU Type"]
            + "<br><b>Processor:</b> "
            + row["Processor Type"]
            + "<br><b>Index:</b> "
            + str(row["Topology Index"])
        )
        return set_label

    def change(change_type):
        if not topo[change_type].any():
            display(Markdown(f"No changes in {change_type}"))
        else:
            display(Markdown(f"**Changes in {change_type}:**"))
            topo_output = topo.loc[topo[change_type] == True]
            topo_output = topo_output["Timestamp"].unique()
            timestamps = []

            for time in topo_output:
                timestamps.append(pd.to_datetime(time).strftime("%m-%d-%y %H:%M:%S"))
            if len(timestamps) < 10:
                display(Markdown(f"{', '.join(timestamps)}"))
            else:
                display(Markdown(f"{len(timestamps)} changes observed"))

    def init_layout(data_fig):
        global df_processed
        data_fig["Affinity Node"] = data_fig["Affinity Node"].astype(str)
        if STISI == "STISI_15_1_3":
            data_fig = data_fig.rename(
                columns={"cp_ci_nl1": "Chip ID", "cp_ci_nl2": "Book ID"}
            ).drop(columns=["cp_ci_nl3"])
            data_fig["Chip ID"] = "Chip: " + data_fig["Chip ID"].astype(str)
            data_fig["Book ID"] = "Book: " + data_fig["Book ID"].astype(str)
            fig = px.treemap(
                data_fig,
                path=["Book ID", "Chip ID", "Label"],
                color=coloring,
                values=None,
                color_discrete_sequence=colors,
            )
        elif STISI == "STISI_15_1_4":
            data_fig = data_fig.rename(
                columns={
                    "cp_ci_nl1": "Chip ID",
                    "cp_ci_nl2": "Node ID",
                    "cp_ci_nl3": "Drawer",
                }
            )
            data_fig["Chip ID"] = "Chip: " + data_fig["Chip ID"].astype(str)
            data_fig["Node ID"] = "Node: " + data_fig["Node ID"].astype(str)
            data_fig["Drawer"] = "Drawer: " + data_fig["Drawer"].astype(str)
            fig = px.treemap(
                data_fig,
                path=["Drawer", "Node ID", "Chip ID", "Label".replace("_", "\n")],
                values=None,
                color=coloring,
                color_discrete_sequence=colors,
            )
        elif STISI == "STISI_15_1_2" or STISI == "STISI_15_1_20":
            data_fig = data_fig.rename(
                columns={"cp_ci_nl2": "Chip ID", "cp_ci_nl1": "Book ID"}
            ).drop(columns=["cp_ci_nl3"])
            data_fig["Chip ID"] = "Chip: " + data_fig["Chip ID"].astype(str)
            data_fig["Book ID"] = "Book: " + data_fig["Book ID"].astype(str)
            fig = px.treemap(
                data_fig,
                path=["Book ID", "Chip ID", "Label"],
                color=coloring,
                color_discrete_sequence=colors,
            )
        else:
            data_fig = data_fig.rename(
                columns={
                    "cp_ci_nl1": "Chip ID",
                    "cp_ci_nl2": "Cluster ID",
                    "cp_ci_nl3": "Drawer",
                }
            )
            data_fig["Chip ID"] = "Chip: " + data_fig["Chip ID"].astype(str)
            data_fig["Cluster ID"] = "Cluster: " + data_fig["Cluster ID"].astype(str)
            data_fig["Drawer"] = "Drawer: " + data_fig["Drawer"].astype(str)
            fig = px.treemap(
                data_fig,
                path=["Drawer", "Cluster ID", "Chip ID", "Label"],
                color=coloring,
                color_discrete_sequence=colors,
            )
        df_processed = data_fig
        df_processed = df_processed.drop(
            columns=[
                "cp_ci_nlinuse",
                "cp_ci_nl4",
                "cp_ci_nl5",
                "speed",
                "wuq_error",
                "honor priority",
                "affinity nodes",
                "topology",
            ]
        )
        fig.data[0].hovertemplate = "%{parent}"
        fig.show()

    if (
        (not topo["speed"].any())
        and (not topo["topology"].any())
        and (not topo["honor priority"].any())
        and (not topo["affinity nodes"].any())
    ):
        display(
            Markdown(
                f"No changes in speed and honor priority. Affinity nodes were not rebuilt"
            )
        )
    else:
        for change_type in changes:
            change(change_type)

    df_filtered["Label"] = df_filtered.apply(lambda row: label(row), axis=1)

    init_layout(df_filtered)